In [1]:
import numpy as np
import pandas as pd

# Рисовать графики сразу же
# %matplotlib inline
import matplotlib.pyplot as plt

# plt.style.use('ggplot')  # Красивые графики
# plt.rcParams['figure.figsize'] = (15, 5)  # Размер картинок
import seaborn as sns

In [29]:
def main_actvts(label_first_text,label_second_text,label_folder_text,label_folder_text_mtrls,hotel_name):
    try:
        # Загрузка данных
        # - Берем "Гость" из файла "Часть 2". Здесь может быть как один человек, так и группа людей, если они вместе заселялись
        # Стараемся каждому гостю из группы сопоставить индивидуальные данные, если они занесены в файл "Часть 1
        # Особенность - по тем или иным причинам имя Гостя может повторяься (Человек дважды за месяц посетил отель, Не сообщили имя второго гостя и вторым именем записано то же самое)
        # Решение - ищем совпадение ф двух файлах по по имени гостя + дата заезда -->
        df_1 = pd.read_excel(label_first_text)
        df_2 = pd.read_excel(label_second_text)

        # Список гостей из файла 2 в историческом порядке каждый в своей строчке
        s_name_2=df_2['Гость']
        name_2=[]
        for i in np.array(s_name_2):
            buf=i.split('\n')
            for j in buf:
                name_2.append(j)

        buf_df_export=pd.DataFrame({'Гость':pd.Series(name_2).str.lstrip('. 123456789').str.rstrip('. 123456789')})

        # Кол-во имен в записи (всего записей 73, а имен 153)
        df_2_columns=np.array(df_2.columns)

        count_guest_name_in_record=[]
        for i in df_2[df_2_columns[0]]:
            count_guest_name_in_record.append(i.count('\n')+1)

        # Каждому гостю ставим дату заезда
        index_counter_for_buf_df_export=0
        for i in range(len(count_guest_name_in_record)):
            buf_df_export.loc[index_counter_for_buf_df_export,'Заезд']=df_2['Заезд'][i]
            index_counter_for_buf_df_export+=count_guest_name_in_record[i]
        buf_df_export=buf_df_export.fillna(method="ffill")
        buf_df_export['Заезд']=buf_df_export['Заезд'].dt.strftime("%d-%m-%Y")    

        # Берем данные из файла часть 1 (инф по кажому гостю)
        df_1_columns=np.array(df_1.columns)

        buf_df_1_export_part = pd.concat([pd.DataFrame(df_1[df_1_columns[0:4]]),#ФИО Гражданство Адрес проживания
                                          pd.DataFrame(df_1[df_1_columns[6:]])],axis=1)#Дата рождения Источник бронирования Сегмент рынка Цель визита

        buf_df_1_export_part=buf_df_1_export_part.rename(columns={'ФИО':'Гость'})
        buf_df_1_export_part['Время заезда']=pd.to_datetime(
                                  buf_df_1_export_part['Заезд'],
                                  format='%d.%m.%y %H:%M').dt.strftime("%H:%M")
        buf_df_1_export_part['Заезд'] = pd.to_datetime(
                                  buf_df_1_export_part['Заезд'],
                                  format='%d.%m.%y %H:%M').dt.strftime("%d-%m-%Y")
        buf_df_1_export_part=buf_df_1_export_part.drop_duplicates(subset=['Гость','Заезд'], keep='first')
        buf=buf_df_1_export_part[buf_df_1_export_part.duplicated(subset=['Гость','Заезд'], keep=False)]

        result_step_1 = pd.merge(buf_df_export, 
                          buf_df_1_export_part,
                          how="left",
                          on=["Гость","Заезд"])

        # 2 Этап
        # Добавляем оставшиеся столбцы  характеризующие сделку из  файла "Часть 2"
        # Оставляю часть которую нужно "присоеденить"
        buf_df_2_export_part=df_2.drop(columns=['Гость','Гражданство'])

        # Убираю заезд чтоб не париться с поядком столбцов. Но в конце заменить!! Тк нет времени часов заезда в df2
        copy_result_step_1=result_step_1.copy()
        result_step_1=result_step_1.drop(columns=['Заезд'])

        # Графики!!!
        citizenship=copy_result_step_1[['Гражданство']]
        citizenship = citizenship.loc[citizenship['Гражданство'] != 'Не указано'].reset_index(drop=True)
        citizenship_drop_russia = citizenship.loc[citizenship['Гражданство'] != 'Российская Федерация'].reset_index(drop=True)
        citizenship_drop_russia=citizenship_drop_russia.rename(columns={'Гражданство':'Гражданство (Без России)'})

        source_cost=buf_df_2_export_part[['Источник','Стоимость']]
        source_nights=buf_df_2_export_part[['Источник','Ночей']]

        df_booking_check_in=buf_df_2_export_part[['Дата бронирования','Заезд']]
        df_booking_check_in['Дата бронирования']=df_booking_check_in['Дата бронирования'].dt.normalize()
        booking_shoulder=np.array((df_booking_check_in['Заезд']-df_booking_check_in['Дата бронирования']).astype('timedelta64[D]'))

        df_booking_check_in=buf_df_2_export_part[['Дата бронирования','Заезд']]
        df_booking_check_in['Дата бронирования']=df_booking_check_in['Дата бронирования'].dt.normalize()
        booking_shoulder=pd.DataFrame((df_booking_check_in['Заезд']-df_booking_check_in['Дата бронирования']).astype('timedelta64[D]')).rename(columns={0:'booking_shoulder'})
        df_booking_check_in=buf_df_2_export_part[['Источник']]
        source_sholder=pd.concat([df_booking_check_in,booking_shoulder],axis=1)

        bins = np.array([0, 1, 5, 10, 20,30,60,90,180])
        x = np.array(source_sholder.booking_shoulder)
        source_sholder['category']=np.digitize(x,bins,right=True)

        def gropped_count(df_get_one_column_gist,col_name_from_df):

            df_buf=df_get_one_column_gist[[col_name_from_df]]
            df_buf=pd.concat([df_buf,pd.DataFrame(np.ones(len(df_buf)),columns=['count'])],axis=1)
            df_buf=df_buf.groupby(col_name_from_df,as_index=False).count()
            return df_buf

        df_booking_check_in=buf_df_2_export_part[['Дата бронирования','Заезд']]
        df_booking_check_in['Дата бронирования']=df_booking_check_in['Дата бронирования'].dt.normalize()
        # Убрала дни, тк важнее месяцы бронирования. В какие месяцы больше броней?
        booking = pd.DataFrame(df_booking_check_in['Дата бронирования'].dt.strftime("%m-%Y"),columns=['Дата бронирования'])   

        booking_grsph=gropped_count(booking,'Дата бронирования')
        check_in_grsph=gropped_count(df_booking_check_in[['Заезд']],'Заезд')

        import locale
        locale.setlocale(locale.LC_ALL, "ru") # задаем локаль для вывода даты на русском языке

        from datetime import date
        import calendar
        from calendar import day_name

        check_in_name_week_day=check_in_grsph.copy()
        for i in range(len(check_in_grsph)):    
            check_in_name_week_day.iloc[i,0]=calendar.day_name[check_in_name_week_day.iloc[i,0].weekday()]

        booking_name_week_day=df_booking_check_in[['Дата бронирования']].copy()
        booking_name_week_day=pd.concat([booking_name_week_day,pd.DataFrame(np.ones(len(booking_name_week_day), dtype=int),columns=['count'])],axis=1)
        for i in range(len(booking_name_week_day)):    
            booking_name_week_day.iloc[i,0]=calendar.day_name[booking_name_week_day.iloc[i,0].weekday()]    

        def get_s_day_result(df_get_one_column_gist,col_name_from_df,title_main):

            df_buf=df_get_one_column_gist.groupby(col_name_from_df,as_index=False).sum()       
            cats = ['понедельник', 'вторник', 'среда', 'четверг', 'пятница', 'суббота', 'воскресенье']
            df_buf[col_name_from_df] = pd.Categorical(df_buf[col_name_from_df], categories=cats, ordered=True)
            df_buf = df_buf.sort_values(col_name_from_df)    
            return df_buf['count'].values
        abc_total_revenue_user = get_s_day_result(check_in_name_week_day,'Заезд','Кол-во заездов по дням недели')
        abc_item_quantity = get_s_day_result(booking_name_week_day,'Дата бронирования','Кол-во броней по дням недели')  

        cities=copy_result_step_1[['Адрес проживания']]
        cities=cities['Адрес проживания'].str.split(',', expand=True)
        cities=cities[cities.columns[0:4]]
        cities=cities.fillna('0')

        for col_name in cities.columns:
            cities[col_name]=cities[col_name].str.strip('г. ')

#         path_to_materials_csv= +'/' #('/'.join(label_folder_text_mtrls.split('/')[0:len(label_folder_text_mtrls.split('/'))-3])) #('/'.join(label_first_text.split('/')[0:len(label_first_text.split('/'))-3]))+
#         data_region = pd.read_csv(path_to_materials_csv+"Перечень регионов РФ.csv")#,sep=';'
#         data_region=data_region[['region']]
#         data_region=pd.concat([data_region,pd.DataFrame(np.ones(len(data_region)),columns=['count'])],axis=1)
#         data_region=data_region.groupby(['region'],as_index=False).count().drop(columns='count')
        
        data_region=pd.read_excel(label_folder_text_mtrls+'\Перечень регионов РФ.xlsx').drop_duplicates()

        df_user_adres=copy_result_step_1[['Гость','Адрес проживания']]
        buf_found_all=pd.DataFrame(columns=['region'])

        regions=np.array(data_region.region, dtype='str')

        for index, row in cities.iterrows():
            for i in np.array(cities.columns):
                # Берем из фрейма с адресами клиентов и идем по строке по одному клиенту
                # Берем первую ячейку первого клиента и провекрям, есть ли такое название города

                # Берем первое слово - область, т.к. иногда встречается такое заполнение (-город) Кемеровская область - Кузбасс 
                buf_region_name=row[i].split(' ')[0]
                mathces_indexes=np.flatnonzero(np.core.defchararray.find(regions,buf_region_name)!=-1)
                # Если для текущего клиента по выбранной ячейке нашли совпадение в столбце региона, то заканчиваем поиск и переходим к др клиенту
                if (len(mathces_indexes)):
                    buf_found_all.loc[index,'region']=data_region.loc[mathces_indexes[0],'region']
                    break

        user_region_result = pd.concat([df_user_adres, buf_found_all], axis=1)

        df_guest_sex=copy_result_step_1[['Гость']]
        df_guest=copy_result_step_1['Гость']
        df_guest=df_guest.str.split(' ', expand=True)
#         path_to_materials_csv=('/'.join(label_first_text.split('/')[0:len(label_first_text.split('/'))-3]))+

        df_russian_names = pd.read_excel(label_folder_text_mtrls+"\Перечень русских имен для определения пола.xlsx")
        df_russian_names=df_russian_names[['Name','Sex']]
        df_russian_names.Name=df_russian_names.Name.str.lower()
        df_russian_names=df_russian_names.drop_duplicates(subset='Name', keep='first')

        df_foreign_names = pd.read_excel(label_folder_text_mtrls+"\Перечень иностранных имен для определения пола.xlsx")
        df_foreign_names=df_foreign_names[['Name','Sex']]
        df_foreign_names=df_foreign_names.replace('Male', 'М')
        df_foreign_names=df_foreign_names.replace('Female', 'Ж')
        df_foreign_names.Name=df_foreign_names.Name.str.lower()
        df_foreign_names=df_foreign_names.drop_duplicates(subset='Name', keep='first')

        for col_name in df_guest.columns:
             df_guest[col_name]=df_guest[col_name].str.lower()

        for col_name in df_guest.columns:
            df_guest=pd.merge(df_guest, df_russian_names,  how='left', left_on=col_name, right_on = 'Name',
                    suffixes=('', str(col_name)+'_ru'))
            df_guest=pd.merge(df_guest, df_foreign_names,  how='left', left_on=col_name, right_on = 'Name',
                    suffixes=('', str(col_name)+'_eng'))

        df_guest=df_guest[df_guest.columns[pd.Series(df_guest.columns).str.contains('Sex').fillna(False)]] 

        sex_rslt=df_guest.iloc[:,0]

        for col_name in df_guest.columns:
            sex_rslt=sex_rslt.fillna(df_guest[col_name])

        user_sex_result=pd.concat([df_guest_sex,pd.DataFrame(sex_rslt,columns=['Sex'])],axis=1)    

        # Доработка excel файла
        # Добавила пол
        copy_result_step_1.insert(1, "Пол", user_sex_result.Sex)
        # Добавила Регион
        copy_result_step_1.insert(4, "Регион РФ", user_region_result.region)
        # Плечо бронирования
        buf_df_2_export_part.insert(4, "Плечо бронирования", booking_shoulder.booking_shoulder.astype('int32',errors='ignore'))
        buf_df_2_export_part=buf_df_2_export_part.drop(columns=['Заезд'])
        buf_df_2_export_part.insert(2, "Месяц бронирования",
                                    pd.to_datetime(
                                    buf_df_2_export_part['Дата бронирования'],
                                    format='%y-%m-%d %H:%M:%s').dt.strftime("%Y%m"))
        buf_df_2_export_part.insert(4, "Время бронирования",
                                    pd.to_datetime(
                                    buf_df_2_export_part['Дата бронирования'],
                                    format='%y-%m-%d %H:%M:%s').dt.strftime("%H:%M"))
        from datetime import date
        import calendar
        from calendar import day_name
        import locale
        locale.setlocale(locale.LC_ALL, "ru") # задаем локаль для вывода даты на русском языке
        buf_df_2_export_part.insert(5, "День недели бронирования",
                                    buf_df_2_export_part['Дата бронирования'].apply(lambda x: calendar.day_name[x.weekday()]))
        buf_df_2_export_part['Дата бронирования']=pd.to_datetime(
                                    buf_df_2_export_part['Дата бронирования'],
                                    format='%y-%m-%d %H:%M:%s').dt.strftime("%Y-%m-%d")

        copy_result_step_1['Заезд']=pd.to_datetime(
                                    copy_result_step_1['Заезд'],
                                    format='%d-%m-%Y').dt.strftime("%Y-%m-%d")
        copy_result_step_1.insert(2, "Месяц заезда",
                                    pd.to_datetime(
                                    copy_result_step_1['Заезд'],
                                    format="%Y-%m-%d").dt.strftime("%Y%m"))
        buf=copy_result_step_1['Время заезда'].copy()
        copy_result_step_1=copy_result_step_1.drop(columns='Время заезда')
        copy_result_step_1.insert(4, "Время заезда",buf)

        copy_result_step_1.insert(5, "День недели заезда",
                                    pd.to_datetime(
                                    copy_result_step_1['Заезд'],
                                    format="%Y-%m-%d").apply(lambda x: calendar.day_name[x.weekday()]))

        hotel_name_s=[]
        for i in range(0,len(copy_result_step_1)):
            hotel_name_s.append(hotel_name)

        copy_result_step_1.insert(0, "Отель",
                                    pd.Series(hotel_name_s))    

        import math
        def myround5(x, base=5):
            return base * round(x/base)
        def myround2(x, base=2):
            return base * round(x/base)

        copy_result_step_1.insert(12, "Возраст шаг 5",
                                    copy_result_step_1['Возраст'].apply(lambda x: 0 if x=='до 1 года'
                                                            else float(x) if math.isnan(float(x))   
                                                            else myround5(float(x)) ))  
        copy_result_step_1.insert(12, "Возраст шаг 2",
                                    copy_result_step_1['Возраст'].apply(lambda x: 0 if x=='до 1 года'
                                                            else float(x) if math.isnan(float(x))   
                                                            else myround2(float(x)) ))   
        import math
        copy_result_step_1['Возраст']=copy_result_step_1['Возраст'].apply(lambda x: 0 if x=='до 1 года'
                                                            else float(x))

        # xlsxwriter - Установить через менеджер библиотек анаконды если код не идет
        import openpyxl
        from openpyxl import load_workbook
        from openpyxl.styles import Border, Side, PatternFill, Font, GradientFill, Alignment

        # ПИшу сюдя то что хочу выгрузить
        result_step_1=copy_result_step_1
        # Create a Pandas Excel writer using XlsxWriter as the engine.
        # path=r"C:\Users\koder\Desktop\проекты\калейдоскоп\KA Turbine/result.xlsx"
        path=label_folder_text+'/'+label_folder_text.split('/')[-1]+' - Отчет.xlsx'
        writer = pd.ExcelWriter(path,engine='xlsxwriter')

        # Сохраняем первыую часть документа и открываем для работы
        result_step_1.to_excel(path, sheet_name='Sheet1',index=False)

        workbook = load_workbook(path)
        worksheet = workbook.active


        #Определили Буквы-имена колонок в экселе. Цикл для каждой колонки под вторую часть
        # i- буква столбца в excel j -кол-во чел вгруппе
        # Заголовки столбцов второй части
        columns_name_part_2=np.array(buf_df_2_export_part.columns)
        counter_columns_name=0
        # Индексация столбоц в экселе
        arr_EN = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z','AA','AB','AC','AD','AE'];

        # Описание стилей для шрифта и границ
        from openpyxl.styles import Font, Color
        thin = Side(border_style="thin", color="000000")
        medium = Side(border_style="medium", color="000000")

        counter_for_border_styl=len(list(buf_df_2_export_part))

        for i in arr_EN[len(list(result_step_1)):len(list(result_step_1))+len(list(buf_df_2_export_part))]:
            # Ставим заголовок из датафрейма в ексель
            worksheet[i+'1'].value = columns_name_part_2[counter_columns_name] 
            # Определение стиля заголовка
            worksheet[i+'1'].font = Font(bold=True)
            # Определение стиля границы заголовка
            worksheet[i+'1'].border = Border(top=thin, left=thin, right=thin, bottom=thin)
            # Начинаем запролнять файл с верху
            index_counter_df=0
            index_counter_excel=2
            # Для каждой определенной группы
            for j in count_guest_name_in_record:
                # Если госте было больше одного
                if j>1:
                    # Объединили ячейку
                    worksheet.merge_cells(i+str(index_counter_excel)+':'+i+str(index_counter_excel+j-1))
                    #Поместили в ячейку нобходимое значение            
                    worksheet[i+str(index_counter_excel)].value = buf_df_2_export_part.iloc[index_counter_df,counter_columns_name]           
                    #Проверка на пропуск даных
                    if (worksheet[i+str(index_counter_excel)].value is np.nan):
                        worksheet[i+str(index_counter_excel)].value="Пропуск данных"       
                        worksheet[i+str(index_counter_excel)].font  = Font(b=True, color="FF0000") 
                    # Сместили индексы
                    index_counter_excel=index_counter_excel+j
                    index_counter_df=index_counter_df+1

                else:
                    #Поместили в ячейку нобходимое значение            
                    worksheet[i+str(index_counter_excel)].value = buf_df_2_export_part.iloc[index_counter_df,counter_columns_name]
                    #Проверка на пропуск даных
                    if (worksheet[i+str(index_counter_excel)].value is np.nan):
                        worksheet[i+str(index_counter_excel)].value="Пропуск данных"       
                        worksheet[i+str(index_counter_excel)].font  = Font(b=True, color="FF0000")            
                    # Сместили индексы (Прописываем через else на случай пропуска данных или 0 гостей)
                    index_counter_excel=index_counter_excel+1
                    index_counter_df=index_counter_df+1            

            # Переходим к следующей колонке
            counter_columns_name=counter_columns_name+1    

        # workbook.save(path)

        #  Проверяем на отсутствие данных первые столбцы в начале
        index_counter_excel=2
        for i in arr_EN[:len(list(result_step_1))]:
            for j in range(2,len(result_step_1)+2):
                if (worksheet[i+str(j)].value is None):
                    worksheet[i+str(j)].value="Пропуск данных"       
                    worksheet[i+str(j)].font  = Font(b=True, color="FF0000")  
                if (worksheet[i+str(j)].value == 'Не указано'):
                    worksheet[i+str(j)].value="Пропуск данных"       
                    worksheet[i+str(j)].font  = Font(b=True, color="FF0000")       
        # Ниже работа со стилем документа excel. Обеспесение аналогичного визуального содержания            
        # Определение стиля границ
        for i in arr_EN[:len(list(result_step_1))+len(list(buf_df_2_export_part))]:
            # Начинаем форматировать по одной ячейке в столбце с верху
            index_counter_excel=2
            # Для каждой определенной группы
            for j in count_guest_name_in_record:
                #Если это  столбец деления (начло чфайла части 2), то нужна жирная левая линия 
                if i==arr_EN[len(list(result_step_1))]:
                    j_cpoy=j
                    while(j_cpoy!=0):
                        worksheet[i+str(index_counter_excel+j_cpoy-1)].border = Border(left=medium)                
                        j_cpoy=j_cpoy-1                

                    #Первая строка в группе
                    worksheet[i+str(index_counter_excel)].border = Border(left=medium,top=medium, bottom=medium,right=None)
                    #Последняя строка в группе
                    worksheet[i+str(index_counter_excel+j-1)].border = Border(left=medium,top=medium, bottom=medium,right=None)

                else:
                    j_cpoy=j
                    while(j_cpoy!=0):
                        worksheet[i+str(index_counter_excel+j_cpoy-1)].border = Border(top=None, bottom=None,right=None,left=None)  
                        j_cpoy=j_cpoy-1

                    #Первая строка в группе
                    worksheet[i+str(index_counter_excel)].border = Border(top=medium, bottom=None,right=None,left=None)
                    #Последняя строка в группе
                    worksheet[i+str(index_counter_excel+j-1)].border = Border(top=None, bottom=medium,right=None,left=None)

                index_counter_excel=index_counter_excel+j        

        # Автомат. устанвка ширины столбца        
        def as_text(value):
            if value is None:
                return ""
            return str(value)

        for column_cells in worksheet.columns:
            length = max(len(as_text(cell.value)) for cell in column_cells)
            worksheet.column_dimensions[arr_EN[int(column_cells[0].column)-1]].width = length

        # Установка фильтра - без преднастроек   
        worksheet.auto_filter.ref = arr_EN[0]+'1:'+arr_EN[len(list(result_step_1))+len(list(buf_df_2_export_part))-1]+'1'        

        workbook.save(path)
        return 1
    except Exception as e: # work on python 3.x
        return str(e)

        

# GUI

In [8]:
# pip install pypiwin32
# pip install pyinstaller

In [32]:
import tkinter as tk
from tkinter import *
from tkinter.messagebox import *
from calendar import month_name
import threading
import time
from tkinter import ttk
from tkinter import Tk, Button, Label
from tkinter import filedialog
from tkinter import HORIZONTAL
from tkinter.messagebox import showinfo
import tkinter as tk
from tkinter import *
from tkinter.messagebox import *
from calendar import month_name
import threading
import time
from tkinter import ttk
from tkinter import Tk, Button, Label
from tkinter import filedialog
from tkinter import HORIZONTAL
from tkinter.messagebox import showinfo


def select_first_file(old_value):
    """Выбор первого файла и вывод пути до него в GUI."""
    new_value = filedialog.askopenfilename()
    if new_value:
        label_fist_file_path['text'] = new_value
        global label_first_text
        label_first_text = new_value
    else:
        label_fist_file_path['text'] = old_value


def select_second_file(old_value):
    """Выбор второго файла и вывод пути до него в GUI."""
    new_value = filedialog.askopenfilename()
    if new_value:
        label_second_file_path['text'] = new_value
        global label_second_text
        label_second_text = new_value
    else:
        label_second_file_path['text'] = old_value

def select_folder(old_value):
    """Выбор паки для графиков и вывод пути до него в GUI."""
    new_value = filedialog.askdirectory()
    if new_value:
        label_folder_file_path['text'] = new_value
        global label_folder_text
        label_folder_text = new_value
    else:
        label_folder_file_path['text'] = old_value

def select_folder_mtrls(old_value):
    """Выбор паки где лежат материалы для проги."""
    new_value = filedialog.askdirectory()
    if new_value:
        label_folder_file_path_mtrls['text'] = new_value
        global label_folder_text_mtrls
        label_folder_text_mtrls = new_value
    else:
        label_folder_file_path_mtrls['text'] = old_value
                
# def select_folder_plots(old_value):
#     """Выбор паки для отчета и вывод пути до него в GUI."""
#     new_value = filedialog.askdirectory()
#     if new_value:
#         label_folder_plots_path['text'] = new_value
#         global label_folder_plots_text
#         label_folder_plots_text = new_value
#     else:
#         label_folder_plots_path['text'] = old_value        

        
def Close():
    root.quit()
    root.destroy()
          
if __name__ == '__main__':
    root = Tk()
    root.title("Формирование отчета")
    # Устновка размеров окна: длина окнаХвысота окна+отступ от края экрана слева при старте+отступ от верхней части экрана
    root.geometry('1000x200+100+50')
    
    
    #------------------------------------
    # create a combobox
    selected_month = tk.StringVar()
    month_cb = ttk.Combobox(root, textvariable=selected_month)

    # get first 3 letters of every month name
    month_cb['values'] = ["Голд", 
                          "Итальянская",
                          "Невский",
                          "Рубинштейна",
                          "Центральный"]
    # prevent typing a value
    month_cb['state'] = 'readonly'
    # place the widget
    month_cb.grid(row=0, column=1)
    
    label_cb = Label(root, text='1.Выбрать название отеля', width=50)
    label_cb.grid(row=0, column=0)

    # bind the selected value changes
    def month_changed(event):
        global hotel_name
        hotel_name = selected_month.get()

    month_cb.bind('<<ComboboxSelected>>', month_changed)
    #------------------------------------
    
    label_first_text = "Путь до файла Гости"
    label_second_text = "Путь до файла Брони"
    label_folder_text = "Путь до папки, куда сохранить отчет"
    label_folder_text_mtrls = "Путь до папки, где лежат Excel файлы с именами"

#     label_folder_plots_text = "Путь до папки, куда сохранить графики"

    # Аргументы класса Button в данном случае: контейнер, в который кладем элемент, текст кнопки, ширина кнопки, лямбда с логикой обработки нажатия кнопки
    button_select_first_file = Button(root, text='2.Выбрать файл Гости', width=40,
                                      command=lambda: select_first_file(label_first_text))
    # Каждый элемент нужно не только положить в контейнер, но и разместить там
    # Есть 3 взаимосключающие в пределах одного контейнера методы: pack, grid, place
    button_select_first_file.grid(row=1, column=0)

    label_fist_file_path = Label(root, text=label_first_text, width=100)
    label_fist_file_path.grid(row=1, column=1)

    button_select_second_file = Button(root, text='3.Выбрать файл Брони', width=40,
                                       command=lambda: select_second_file(label_second_text))
    button_select_second_file.grid(row=2, column=0)

    label_second_file_path = Label(root, text=label_second_text, width=100)
    label_second_file_path.grid(row=2, column=1)
    # Выбор папки для отчета
    button_select_folder = Button(root, text='4.Выбрать папку для отчета', width=40,
                                       command=lambda: select_folder(label_folder_text))
    button_select_folder.grid(row=3, column=0)

    label_folder_file_path = Label(root, text=label_folder_text, width=100)
    label_folder_file_path.grid(row=3, column=1)    
    
    # Выбор папки c материалом для кода
    button_select_folder_mtrls = Button(root, text='5.Выбрать с материалами для кода', width=40,
                                       command=lambda: select_folder_mtrls(label_folder_text_mtrls))
    button_select_folder_mtrls.grid(row=4, column=0)

    label_folder_file_path_mtrls = Label(root, text=label_folder_text_mtrls, width=100)
    label_folder_file_path_mtrls.grid(row=4, column=1)     
    
   

    def run_thread():
        execute_thread = threading.Thread(target=print_cf)
        execute_thread.start()
        
    def print_cf(event = None):
        progress.start()
#         time.sleep(3)
        f=main_actvts(label_first_text,label_second_text,label_folder_text,label_folder_text_mtrls,hotel_name)       
        progress.stop()
        go_button['state'] = tk.DISABLED
        if f==1:
            messagebox.showinfo("Сообщение", "Отчет готов!")
        else:     
            messagebox.showerror("Ошибка", "Отчет НЕ готов! \n"+f)
    # Button for closing
    go_button = Button(root, text="6.Выполнить", width=40, command=run_thread)
    go_button.grid(row=5, column=0)
    
    progress = ttk.Progressbar(root, orient = HORIZONTAL, length = 100, mode = 'indeterminate')
    progress.grid(row=5, column=1)
    
    close_button = Button(root, text="7.Закрыть", width=40, command=Close)
    close_button.grid(row=7, column=0)

    # Запуск графического интерфейса
    root.focus()
    #root.iconbitmap(r'C:\Users\koder\kaleidoskop_2022\kaleidoscope.ico')
    root.mainloop()
